# Clustering Crypto

In [2]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Fetching Cryptocurrency Data

In [3]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"
response_data = requests.get(url)
data = response_data.json()

In [5]:
# Create a DataFrame 
# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame.
crypto_df = pd.DataFrame.from_dict(data['Data']).transpose()
crypto_df.head()

,Id,Url,ImageUrl,ContentCreatedOn,Name,Symbol,CoinName,FullName,Algorithm,ProofType,...,SortOrder,Sponsored,Taxonomy,Rating,IsTrading,TotalCoinsMined,BlockNumber,NetHashesPerSecond,BlockReward,BlockTime
42,4321,/coins/42/overview,/media/35650717/42.jpg,1427211129,42,42,42 Coin,42 Coin (42),Scrypt,PoW/PoS,...,34,False,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...","{'Weiss': {'Rating': '', 'TechnologyAdoptionRa...",True,42,195131,0,0,0
300,749869,/coins/300/overview,/media/27010595/300.png,1517935016,300,300,300 token,300 token (300),N/A,N/A,...,2212,False,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...","{'Weiss': {'Rating': '', 'TechnologyAdoptionRa...",True,300,0,0,0,0
365,33639,/coins/365/overview,/media/352070/365.png,1480032918,365,365,365Coin,365Coin (365),X11,PoW/PoS,...,916,False,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...","{'Weiss': {'Rating': '', 'TechnologyAdoptionRa...",True,NaN,NaN,NaN,NaN,NaN
404,21227,/coins/404/overview,/media/35650851/404-300x300.jpg,1466100361,404,404,404Coin,404Coin (404),Scrypt,PoW/PoS,...,602,False,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...","{'Weiss': {'Rating': '', 'TechnologyAdoptionRa...",True,1.20127e+06,44915,0,15.4347,60
433,926547,/coins/433/overview,/media/34836095/433.png,1541597321,433,433,433 Token,433 Token (433),N/A,N/A,...,3505,False,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...","{'Weiss': {'Rating': '', 'TechnologyAdoptionRa...",False,1.12518e+08,10821438,231060807658772,2,0


### Data Preprocessing

In [6]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
crypto_df = crypto_df[['CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply']]
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,42,42
300,300 token,N/A,True,N/A,300,300
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.20127e+06,532000000
433,433 Token,N/A,False,N/A,1.12518e+08,1000000000


In [7]:
# Keep only cryptocurrencies that are trading
crypto_df = crypto_df.loc[crypto_df['IsTrading'] == True]
crypto_df.IsTrading.value_counts()

True    4173
Name: IsTrading, dtype: int64

In [8]:
# Keep only cryptocurrencies with a working algorithm
crypto_df = crypto_df.loc[crypto_df['Algorithm'] != 'N/A']
crypto_df.Algorithm.value_counts()

Scrypt               452
X11                  219
SHA-256              154
PoS                   71
X13                   70
                    ... 
IMesh                  1
CryptoNote             1
Cryptonight-GPU        1
CryptoNight Heavy      1
ProgPowZ               1
Name: Algorithm, Length: 121, dtype: int64

In [9]:
# Remove the 'IsTrading' column
crypto_df = crypto_df.drop(columns=['IsTrading'])
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,42,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.20127e+06,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0,0


In [10]:
# Remove rows with at least 1 null value
crypto_df = crypto_df.dropna()
crypto_df.head()


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,42,42
404,404Coin,Scrypt,PoW/PoS,1.20127e+06,532000000
808,808,SHA-256,PoW/PoS,0,0
1337,EliteCoin,X13,PoW/PoS,2.94803e+10,314159265359
BTCD,BitcoinDark,SHA-256,PoW/PoS,1288862,22000000


In [11]:
# Remove rows with cryptocurrencies having no coins mined
crypto_df = crypto_df.loc[crypto_df['TotalCoinsMined'] > 0]
crypto_df.TotalCoinsMined.min()

41.99995243

In [12]:
# Drop rows where there are 'N/A' text values
crypto_df = crypto_df.replace('N/A', float("NaN"))
crypto_df =  crypto_df.dropna()

In [13]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df 
coinName_df = crypto_df[['CoinName']]
coinName_df.head()

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTCD,BitcoinDark
XPY,PayCoin


In [14]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_df = crypto_df.drop(columns=['CoinName'])
crypto_df.head()


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.201270e+06,532000000
1337,X13,PoW/PoS,2.948032e+10,314159265359
BTCD,SHA-256,PoW/PoS,1.288862e+06,22000000
XPY,SHA-256,PoS,1.199533e+07,12500000


In [15]:
# Create dummy variables for text features
crypto_df_encoded = pd.get_dummies(crypto_df, columns=['Algorithm', 'ProofType'])
crypto_df_encoded.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_BMW512 / Echo512,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.201270e+06,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.948032e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTCD,1.288862e+06,22000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
XPY,1.199533e+07,12500000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
# Standardize data
scaler = StandardScaler()
scaler.fit(crypto_df_encoded[['TotalCoinsMined','TotalCoinSupply']])
crypto_df_encoded[['TotalCoinsMined','TotalCoinSupply']] = scaler.transform(crypto_df_encoded[['TotalCoinsMined','TotalCoinSupply']])
crypto_df_encoded.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_BMW512 / Echo512,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,-0.118800,-0.156186,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,-0.118774,-0.148291,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,0.529303,4.506026,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTCD,-0.118772,-0.155859,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
XPY,-0.118537,-0.156000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Reducing Dimensions Using PCA

In [17]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(crypto_df_encoded)

In [18]:
# Create a DataFrame with the principal components data
pca_columns = ["principal component 1", "principal component 2", "principal component 3"]
df_crypto_pca = pd.DataFrame(
    data=crypto_pca, columns=pca_columns
)
df_crypto_pca.head()

,principal component 1,principal component 2,principal component 3
0,-0.212886,-0.783651,0.167841
1,-0.207292,-0.782931,0.173305
2,3.544238,-0.397590,2.897783
3,-0.214525,-0.672917,0.069128
4,-0.203280,-0.108508,-0.129340


### Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [19]:
!pip install -U altair
import altair as alt

     |████████████████████████████████| 727 kB 5.1 MB/s eta 0:00:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [20]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(df_crypto_pca)
    inertia.append(km.inertia_)


# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
alt.Chart(df_elbow).mark_line().encode(
    x='k',
    y='inertia'
)



alt.Chart(...)

Running K-Means with `k=<your best value for k here>`

In [21]:
# Initialize the K-Means model
model = KMeans(n_clusters=4, random_state=0)
# Fit the model
model.fit(df_crypto_pca)
# Predict clusters
predictions = model.predict(df_crypto_pca)
# Create a new DataFrame including predicted clusters and cryptocurrencies features
clustered_df = pd.concat([coinName_df,crypto_df],axis=1)
clustered_df["class"] = model.labels_
clustered_df = pd.concat([clustered_df.reset_index(),df_crypto_pca.reset_index().drop(columns='index')],axis=1)
clustered_df = clustered_df.set_index('index')
clustered_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,class,principal component 1,principal component 2,principal component 3
index,,,,,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42,0,-0.212886,-0.783651,0.167841
404,404Coin,Scrypt,PoW/PoS,1.201270e+06,532000000,0,-0.207292,-0.782931,0.173305
1337,EliteCoin,X13,PoW/PoS,2.948032e+10,314159265359,0,3.544238,-0.397590,2.897783
BTCD,BitcoinDark,SHA-256,PoW/PoS,1.288862e+06,22000000,0,-0.214525,-0.672917,0.069128
XPY,PayCoin,SHA-256,PoS,1.199533e+07,12500000,0,-0.203280,-0.108508,-0.129340


### Visualizing Results

#### 3D-Scatter with Clusters

In [25]:
# Create a 3D-Scatter with the PCA data and the clusters
alt.Chart(clustered_df).mark_circle(size=60).encode(
    x='principal component 1',
    y='principal component 2',
    color='class',
    tooltip=["CoinName", "Algorithm", "TotalCoinsMined", "TotalCoinSupply"]
).interactive()





alt.Chart(...)

#### Scatter Plot with Tradable Cryptocurrencies

In [31]:
# Scale data to create the scatter plot
scaler = StandardScaler()
scaler.fit(clustered_df[['TotalCoinsMined','TotalCoinSupply']])
clustered_df[['TotalCoinsMined','TotalCoinSupply']] = scaler.transform(clustered_df[['TotalCoinsMined','TotalCoinSupply']])
clustered_df.head()


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,class,principal component 1,principal component 2,principal component 3
index,,,,,,,,,
42,42 Coin,Scrypt,PoW/PoS,-0.118800,-0.156186,0,-0.212886,-0.783651,0.167841
404,404Coin,Scrypt,PoW/PoS,-0.118774,-0.148291,0,-0.207292,-0.782931,0.173305
1337,EliteCoin,X13,PoW/PoS,0.529303,4.506026,0,3.544238,-0.397590,2.897783
BTCD,BitcoinDark,SHA-256,PoW/PoS,-0.118772,-0.155859,0,-0.214525,-0.672917,0.069128
XPY,PayCoin,SHA-256,PoS,-0.118537,-0.156000,0,-0.203280,-0.108508,-0.129340


In [33]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"

alt.Chart(clustered_df).mark_circle(size=60).encode(x="TotalCoinsMined", y="TotalCoinSupply", color='class')

alt.Chart(...)

In [36]:
display(clustered_df)

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,class,principal component 1,principal component 2,principal component 3
index,,,,,,,,,
42,42 Coin,Scrypt,PoW/PoS,-0.118800,-0.156186,0,-0.212886,-0.783651,0.167841
404,404Coin,Scrypt,PoW/PoS,-0.118774,-0.148291,0,-0.207292,-0.782931,0.173305
1337,EliteCoin,X13,PoW/PoS,0.529303,4.506026,0,3.544238,-0.397590,2.897783
BTCD,BitcoinDark,SHA-256,PoW/PoS,-0.118772,-0.155859,0,-0.214525,-0.672917,0.069128
XPY,PayCoin,SHA-256,PoS,-0.118537,-0.156000,0,-0.203280,-0.108508,-0.129340
...,...,...,...,...,...,...,...,...,...
BTC,Bitcoin,SHA-256,PoW,-0.118394,-0.155874,2,-0.183921,0.700158,-0.154002
ETH,Ethereum,Ethash,PoW,-0.116327,-0.156186,2,-0.176805,0.698745,-0.138547
WAVES,Waves,Leased POS,LPoS,-0.116602,-0.154702,0,-0.182064,-0.062515,-0.059791
